In [ ]:
import pandas as pd
from bs4 import BeautifulSoup
import requests
import re
import datetime
from difflib import SequenceMatcher
import pymysql
from sqlalchemy import create_engine
from tqdm import tqdm
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from tqdm import tqdm
import time

In [ ]:
def db_connection(host_name='ds'):
    host_url = "db.ds.mycelebs.com"
    user_nm = 
    passwd = 
    port_num = 3306
    db_name = "star_ko"
    conn = pymysql.connect(host=host_url, user=user_nm, passwd=passwd, port = port_num, charset='utf8',
                           db=db_name, cursorclass=pymysql.cursors.DictCursor)
    # cursorclass=pymysql.cursors.DictCursor 추가 -> 데이터프레임으로 다루기 쉽게 딕셔너리 형태로 결과 반환해줌, cursor는 튜플형태
    # db=db,
    engine = create_engine(f'mysql+pymysql://{user_nm}:{passwd}@{host_url}:{port_num}/{db_name}?charset=utf8mb4')
    return conn, engine

## 1.1 수기 매핑 전에

> 수기 매핑해야 하는 개체들 : vibecode가 null이거나 vibecode가 중복인 개체들
- 수기로 하기 전에 다른 루트로 vibecode를 다시 한번 찾아주는 작업임
- 먼저 해당 개체가 그룹 멤버인지를 크롤링해서 살펴보고, 그룹에 속해있지 않으면 패스, 속해있으면 그룹명 기억해뒀다가,
- 해당 개체 이름으로 바이브에서 서치, 한명씩 들어가서 발매한 앨범의 아티스트명 크롤링
- 아티스트명과 기억해둔 그룹명이 일치하다면 매핑하는 방식임
<br>
  
- 예를 들어, 태연의 vibecode가 필요하다고 가정
- 태연의 네이버 프로필에서 그룹 크롤링 -> '소녀시대'를 기억
- VIBE에서 태연 검색
- 모든 '태연'에 들어가서 해당 '태연'이 발매한 앨범의 아티스트 모두 크롤링 ex. 태연 / 이승철, 태연 / 소녀시대 / 소녀시대-태티서 ...
- '소녀시대'를 발견하면 해당 '태연'의 vibecode를 매핑

In [ ]:
conn, engine = db_connection()

qry = f'SELECT * FROM star_album_genre WHERE vibeCd is NULL'
nullData = pd.read_sql(qry, con=conn)
nullData_list = nullData['series_id'].to_list()

qry = f'SELECT * FROM star_album_genre GROUP BY vibeCd HAVING COUNT(vibeCd) > 1'
dupData = pd.read_sql(qry, con=conn)
dupData_list = dupData['series_id'].to_list()

conn.close()

In [ ]:
handmapping = []

handmapping = nullData_list + dupData_list
print("nullData_list : ", len(nullData_list), "dupData_list : ", len(dupData_list), "합 : ", len(handmapping))

In [ ]:
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless')
driver = webdriver.Chrome(ChromeDriverManager().install(), options=chrome_options)

conn, engine = db_connection()
cursor = conn.cursor(pymysql.cursors.DictCursor)

sucess = []
index_err = []
null_err = []

series_id = name = naver_url = artist_url = album_url = vibeCd = albumName = vibeLink = artist = ''
 
for idx in tqdm(handmapping) :

    qry = f'SELECT * FROM star_ko_profile_url WHERE series_id={idx}'
    profileData = pd.read_sql(qry, con=conn)
    
    try : 

        series_id = profileData['series_id'][0]
        name = profileData['name'][0]
        people_id = profileData['naver_people_id'][0]

        if pd.isnull(people_id) :
            qry3 = f"UPDATE star_album_genre SET vibeCd=-1 WHERE series_id={series_id}"
            cursor.execute(qry3)
            conn.commit()
        else :
            url = f'https://search.naver.com/search.naver?where=nexearch&sm=tab_etc&mra=bjky&pkid=1&os={people_id}&qvt=0&query={name}'

            print("이름: ", name, "series_id: ", series_id, "url: ", url)

            time.sleep(0.5)

            naver_html = requests.get(url).text
            naver_soup = BeautifulSoup(naver_html, 'html.parser')
            time.sleep(0.5)

            belong_group = ''

            try :

                dt_in_info_txt = naver_soup.select('div[class="detail_info"]')[0].select('dt')
                dd_in_info_txt = naver_soup.select('div[class="detail_info"]')[0].select('dd')

                for row_len in range(len(dt_in_info_txt)):
                    title = dt_in_info_txt[row_len].text
                    inner = dd_in_info_txt[row_len]

                    if '그룹' in title:
                        belong_group = inner.text
                        belong_group = belong_group.split(', ')[0]
                        print(belong_group)

                        driver.get(f'https://vibe.naver.com/search/artists?query={name}')

                        try :
                            driver.find_element_by_xpath('//*[@id="app"]/div[2]/div/div/a[2]').click()
                        except :
                            pass

                        time.sleep(5)
                        source = driver.page_source
                        soup = BeautifulSoup(source, "lxml")
                        time.sleep(3)

                        # 바이브에서 모든 동명이인 리스트
                        sub_artist_url = []

                        sub_list = soup.select('div[class="sub_list"]')[0].select('li')
                        for sub in range(len(sub_list)) :
                            sub_artist_url_ = sub_list[sub].select('a')[0].get('href')
                            sub_artist_url_ = 'https://vibe.naver.com' + sub_artist_url_
                            sub_artist_url.append(sub_artist_url_)

                        try :
                            for v in sub_artist_url :
                                driver.get(v)

                                time.sleep(1)

                                source_ = driver.page_source
                                soup_ = BeautifulSoup(source_, "lxml")

                                # 한명 한명의 앨범 아티스트 리스트
                                artist_name_total = []

                                artist_list = soup_.select('div[class="artist"]')
                                for l in artist_list :
                                    try : 
                                        artist_name = l.select('span[class="text"]')[0].text
                                        in_bracket = re.findall('\(([^)]+)', artist_name)  # 괄호 안 ex. 샤이니(SHINee) -> SHINee
                                        artist_name_total = artist_name_total + in_bracket

                                        out_bracket = re.sub(r'\([^)]*\)', '', artist_name)  # 괄호 밖 ex. 샤이니(SHINee) -> 샤이니
                                        out_bracket = out_bracket.rstrip()
                                        artist_name_total.append(out_bracket)

                                        artist_name_total = set(artist_name_total)
                                        artist_name_total = list(artist_name_total)  # ['샤이니', 'SHINee']
                                    except :
                                        pass

                                # 아티스트 리스트에 그룹명이 있다면 매핑
                                if belong_group in artist_name_total :
                                    current_url = driver.current_url
                                    vibeCd = re.findall("\d+", current_url)[0]

                                    qry2 = f"UPDATE star_album_genre SET vibeCd={vibeCd} WHERE series_id={series_id}"
                                    cursor.execute(qry2)
                                    conn.commit()

                                    print("[INFO]", name, " update 완료")
                                    sucess_ = {"name":name, "series_id":series_id, "vibeCd":vibeCd}
                                    sucess.append(sucess_)

                                    raise NotImplementedError

                        except :
                            pass

            except IndexError :
                print("[INFO] IndexError 발생!!!!!!!!")
                print("[INFO] 스타의 profile url을 확인하세요. ")
                index_err_ = {"name":name, "series_id":series_id}
                index_err.append(index_err_)

                pass
            
    # 프로필 row가 아예 없는 경우        
    except :
        null_err_ = {"name":name, "series_id":series_id}
        null_err.append(null_err_)

driver.close()
conn.close()

In [ ]:
# 코드 돌린 후

conn, engine = db_connection()

qry = f'SELECT * FROM star_album_genre WHERE vibeCd is NULL'
nullData = pd.read_sql(qry, con=conn)
nullData_list = nullData['series_id'].to_list()

qry = f'SELECT * FROM star_album_genre GROUP BY vibeCd HAVING COUNT(vibeCd) > 1'
dupData = pd.read_sql(qry, con=conn)
dupData_list = dupData['series_id'].to_list()

conn.close()

In [ ]:
handmapping = []

handmapping = nullData_list + dupData_list
print("nullData_list : ", len(nullData_list), "dupData_list : ", len(dupData_list), "합 : ", len(handmapping))

## 1.2 수기 매핑 (중복값 & null 값)

In [ ]:
conn, engine = db_connection()
cursor = conn.cursor(pymysql.cursors.DictCursor)

chrome_options = webdriver.ChromeOptions()
# chrome_options.add_argument('--headless')
driver = webdriver.Chrome(ChromeDriverManager().install(), options=chrome_options)
driver.get('https://vibe.naver.com/search?query=')

naver_driver = webdriver.Chrome(ChromeDriverManager().install(), options=chrome_options)

for idx in tqdm(handmapping) :
    qry = f'SELECT series_id, cd_name, cd_real_name, cd_category, cd_gender, cd_group, cd_solr_search, cd_height, cd_weight, cd_birth, cd_agency, cd_debut, cd_academic, cd_career, cd_award FROM star_ko_data WHERE series_id={idx}'
    result = pd.read_sql(qry, conn)
    display(result)
    
    qry2 = f'SELECT * FROM star_ko_profile_url WHERE series_id={idx}'
    result2 = pd.read_sql(qry2, conn)
    series_id = result2['series_id'][0]
    name = result2['name'][0]
    naver_people_id = result2['naver_people_id'][0]
    url = f'https://search.naver.com/search.naver?where=nexearch&sm=tab_etc&mra=bjky&pkid=1&os={naver_people_id}&qvt=0&query={name}'
    
    naver_driver.get(url)
    
    driver.get(f'https://vibe.naver.com/search/artists?query={name}')

    
    try :
        driver.find_element_by_xpath('//*[@id="app"]/div[2]/div/div/a[2]').click()
    except :
        pass
    
    vibe = input("vibeCd를 입력하세요. ")
    
    if vibe == '' :
        vibeCd = -1
    else :
        vibeCd = vibe
        
    updateQRY = f'UPDATE star_album_genre SET vibeCd={vibeCd} WHERE series_id={series_id}'
    cursor.execute(updateQRY)
    conn.commit()